In [30]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

h = {"User-Agent": "Mozilla/5.0..."}

res = req.get('https://www.melon.com/mymusic/dj/mymusicdjplaylistview_inform.htm?plylstSeq=494499914', headers = h)

res.text # html

soup = bs(res.text, 'lxml')

# 노래제목
song = soup.select('div.ellipsis.rank01 >span >a')

# 노래가수
singer = soup.select('div.ellipsis.rank02 >a')

# `.button_etc.like` 클래스를 가진 모든 <button> 태그를 선택
buttons = soup.select('button.button_etc.like')

# 첫 번째와 마지막 두 개의 태그를 제외
selected_buttons = buttons[1:-2]

# 남은 태그들에서 `data-song-no` 속성의 값을 추출
song_numbers = [btn['data-song-no'] for btn in selected_buttons]



songList = []
singerList = []
songNumList = []
rankList = []

for i in range(len(song)):
    songList.append(song[i].text)
    singerList.append(singer[i].text)
    songNumList.append(song_numbers[i])
    rankList.append(i+1)

import pandas as pd
dic = {'번호':rankList,'제목': songList, '가수':singerList, '곡코드번호':songNumList}
melon = pd.DataFrame(dic)

melon.set_index('번호', inplace=True)

melon.to_csv('아이유신나는곡들.csv', encoding='utf-8')




In [32]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd

melon = pd.read_csv('아이유신나는곡들.csv', index_col='번호')

browser = webdriver.Chrome()  

lyricsList = []

for num in melon['곡코드번호']:
    browser.get(f'https://www.melon.com/song/detail.htm?songId={num}')
    lyric = browser.find_element(By.CLASS_NAME, "lyric")
    lyricsList.append(lyric.text)

browser.quit()

melon["가사"] = lyricsList

melon.to_csv('아이유_곡_가사.csv', encoding='utf-8')
